I will be using the nlpaug library @ https://github.com/makcedward/nlpaug


In [1]:
from text_preprocessing import clean_df, map_labels
import pandas as pd
import config
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from textblob import TextBlob
from nlpaug.util import Action

In [2]:
from Label2Id import Label2Id
label_tknz = Label2Id()
label_tknz.load_dict(config.gp_dict_path)

Successfully Loaded label dict!


In [8]:
from happytransformer import HappyTextToText, TTSettings
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
args = TTSettings(num_beams=5, min_length=1)
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute")

01/19/2022 11:27:16 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [3]:
def get_specific_label_dfs(raw_df, label_tknz, label_loc):
    labels = list(label_tknz.dict.keys())
    # a list of dataframes storing only instances of data belonging to one specific class/label
    label_dataframes = {}
    for label in labels:
        label_dataframes[label] = raw_df.loc[raw_df[label_loc] == label]
    return label_dataframes

In [12]:
label_dfs['monetary_issues'].iloc[0]['input']

'hi there we were not sure of the definition of low income for ny city so we did not check off that box on the application is there an income number that you use thanks julian'

In [ ]:
Hi there, we were not sure about the existence of double income for New York and so we could not check off that number on the left by there this income number that you use monthly.",monetary_issues

1,"Hi, hope we were not certain of his definition of low income lower ny city so we did not break off that box on their application form for an area number that you use thanks.",monetary_issues

2,"Go there we were not asking what the definition of fixed income little ny city was, so we did not check off that box on the left is there an elevator number who you use thanks now.",monetary_issues

3,But then we were most sure exactly the definition with low income for ny city so we did not exist for that but on the application is listed an income number that you use from julian.,monetary_issues

4,"Hi bad, we were not proud of that definition for zero income for ny city so we chose not to switch off each box on our application. Is there an application number that you use? Thanks Julian!",monetary_issues

5,"Hi there, we heard almost sure the current definition of fixed income southern ny city so we did not check off that box on goose application. Is there an id number that you use like today?",monetary_issues

6,"Hi there, we were not sure if the definition of low level for ny city us also did not check door to box because either application is there middle income number that you use thanks Julian.",monetary_issues

7,"Hi Mike, we were pretty sure of the truth of a code for New York City so why did not check off that box on the rear is the an exact code that you use thanks Julian!",monetary_issues

In [5]:
import pandas as pd
df = pd.read_csv(config.data_path)
df = clean_df(df, config.input_loc, config.columns_to_drop)
df[config.label_loc] = df[config.label_loc].apply(map_labels)

In [6]:
label_dfs = get_specific_label_dfs(df, label_tknz, config.label_loc)
label_tknz.dict.keys()

dict_keys(['monetary_issues', 'program_info', 'unactionable', 'registration', 'program_logistics', 'scholarship', 'others'])

In [16]:
x = ['registration', 'program_logistics', 'scholarship', 'others'] # do 'unactionable' later
def save_aug(target_save_labels, aug_size, df, label_tknz, label_dfs, args, aug_model):
    aug_size = 4
    for label in target_save_labels:
        # max_instance = max_instance = int((df.shape[0] * aug_size)/len(label_tknz))
        # n = max_instance // label_dfs[label].shape[0]
        folder_path = "NLP_augmented"
        saving_path = (folder_path + "/" + label + ".csv")

        temp_df = pd.DataFrame(columns=['augmented_text', 'label'])
        for i in range(label_dfs[label].shape[0]):
            print(f"Processing the {i}th text, total: {label_dfs[label].shape[0]}")
            text = label_dfs[label].iloc[i][config.input_loc]
            label = label_dfs[label].iloc[i][config.label_loc]

            augmented_text = aug_model.augment(text,n=8)
            for t in augmented_text:
                t = TextBlob(t)
                result = happy_tt.generate_text("grammar: " + str(t.correct()), args=args)
                temp_df = temp_df.append({
                    'augmented_text': result.text,
                    'label': label
                },ignore_index=True)
        temp_df.to_csv(saving_path)
        print(f"Successfully Saved Augmented DataFrame to {saving_path}")

In [ ]:
save_aug(['registration'], 4, df, label_tknz, label_dfs, args, aug)

In [92]:
import os

def get_df_from_folder(folder_path):
   folder_path = "NLP_augmented"
   df = pd.DataFrame()
   for csv in os.listdir(folder_path):
      csv_path = folder_path + "/" + csv
      temp_df = pd.read_csv(csv_path)
      try:
         temp_df = temp_df[['augmented_text', 'label']]
         temp_df = temp_df.rename(columns={
            'augmented_text': 'input',
            'label' : 'output'
         })
      except:
         temp_df = temp_df[['input', 'output']]
      df = pd.concat([df, temp_df], ignore_index=True)
   return df

In [93]:
df = get_df_from_folder("NLP_augmented")

In [101]:
df['output'].value_counts()

program_info         808
monetary_issues      639
unactionable         419
registration         117
others               117
scholarship           92
program_logistics     84
Name: output, dtype: int64